In [2]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [3]:
from flopco import FlopCo

In [4]:
import torch
from torchvision.models import resnet18

from tqdm import tqdm
import numpy as np
import os
import random

from aimet_torch.model_preparer import prepare_model
from aimet_torch.batch_norm_fold import fold_all_batch_norms
from aimet_common.defs import QuantScheme
from aimet_torch.quantsim import QuantizationSimModel
from aimet_torch.adaround.adaround_weight import Adaround, AdaroundParameters

from source.data import get_imagenet_test_loader, get_imagenet_train_val_loaders
from source.eval import accuracy
from source.admm import build_cp_layer
from source.utils import bncalibrate_model
from source.rank_map import get_rank_map


seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

2023-01-16 18:10:28,172 - root - INFO - AIMET


In [5]:
def pass_calibration_data(sim_model, use_cuda):
    batch_size = train_loader.batch_size

    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    samples = 1000

    batch_cntr = 0
    with torch.no_grad():
        for input_data, target_data in train_loader:

            inputs_batch = input_data.to(device)
            sim_model(inputs_batch)

            batch_cntr += 1
            print(batch_cntr * batch_size)
            if (batch_cntr * batch_size) >= samples:
                break

In [6]:
train_loader, val_loader = get_imagenet_train_val_loaders(data_root='/gpfs/gpfs0/k.sobolev/ILSVRC-12/',
                                       batch_size=500,
                                       num_workers=4,
                                       pin_memory=True,
                                       val_perc=0.04,
                                       shuffle=True,
                                       random_seed=seed)

In [7]:
test_loader = get_imagenet_test_loader(data_root='/gpfs/gpfs0/k.sobolev/ILSVRC-12/', 
                                       batch_size=500,
                                       num_workers=4,
                                       pin_memory=True,
                                       shuffle=False)

In [8]:
method = 'admm'
qscheme = 'tensor_affine'
bits = 8
eps = 0.003
decomp = 'cp3'
rank_map = get_rank_map(eps, decomp)
num_samples = 10000

In [9]:
# model = resnet18(pretrained=True).to(device)
print(f"loading model e={eps}_d={decomp}_{qscheme.split('_')[-1]}.calibrated_{num_samples}")
model = torch.load(f"e={eps}_d={decomp}_{qscheme.split('_')[-1]}.calibrated_{num_samples}")
model.eval()
model = model.to(device)

loading model e=0.003_d=cp3_affine.calibrated_10000


In [11]:
model_stats = FlopCo(model.to(device), img_size=(1, 3, 224, 224), device=device)
orig_macs = 0
for x in model_stats.macs.values():
    orig_macs += x[0]
orig_macs

553766100

In [9]:
for module in ['layer1', 'layer2', 'layer3', 'layer4']:
    for layer_path in [f'{module}.0.conv1', f'{module}.0.conv2', 
#                        f'{module}.0.downsample',
                       f'{module}.1.conv1', f'{module}.1.conv2']:
        # there is no layer1.0.downsample layer
        if layer_path == 'layer1.0.downsample': continue
            
        lname, lidx, ltype = layer_path.split('.')
        lidx = int(lidx)
        layer = model.__getattr__(lname)[lidx].__getattr__(ltype)
        kernel_size = layer.kernel_size
        stride = layer.stride
        padding = layer.padding
        cin = layer.in_channels
        cout = layer.out_channels
        rank = rank_map[layer_path]
        bias = layer.bias
        if bias is not None: bias = bias.detach()
        
        print(f'loading {bits}bit_{qscheme}/factors_{method}_seed{seed}/{layer_path}_{method}_random_rank_{rank}')
        A = torch.load(f'../{bits}bit_{qscheme}/factors_{method}_seed{seed}/{layer_path}_{method}_random_rank_{rank}_mode_0.pt').to(device)
        assert A.dtype == torch.float 
        B = torch.load(f'../{bits}bit_{qscheme}/factors_{method}_seed{seed}/{layer_path}_{method}_random_rank_{rank}_mode_1.pt').to(device)
        C = torch.load(f'../{bits}bit_{qscheme}/factors_{method}_seed{seed}/{layer_path}_{method}_random_rank_{rank}_mode_2.pt').to(device)

        model.__getattr__(lname)[lidx].__setattr__(
            ltype, build_cp_layer(rank, [A,B,C], bias, cin, cout, kernel_size, padding, stride).to(device))

AttributeError: 'Sequential' object has no attribute 'kernel_size'

In [10]:
model_stats = FlopCo(model.to(device), img_size=(1, 3, 224, 224), device=device)
redc_macs = 0
for x in model_stats.macs.values():
    redc_macs += x[0]
redc_macs / orig_macs

0.39531666421972406

In [11]:
accuracy(model, test_loader, device=device)

100%|██████████| 100/100 [01:48<00:00,  1.09s/it]


0.42276

In [11]:
num_samples = 10000
model = bncalibrate_model(model, train_loader, num_samples=num_samples, device=device)

  1%|          | 21/2459 [01:51<3:34:47,  5.29s/it]


In [9]:
accuracy(model, test_loader, device=device)

100%|██████████| 100/100 [01:47<00:00,  1.08s/it]


0.59094

In [20]:
torch.save(model, f"e={eps}_d={decomp}_{qscheme.split('_')[-1]}.calibrated_{num_samples}")

# Quantization

In [10]:
model = prepare_model(model)

2023-01-16 17:54:15,004 - Quant - INFO - Functional         : Adding new module for node: {add} 
2023-01-16 17:54:15,005 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_0_relu_1} 
2023-01-16 17:54:15,006 - Quant - INFO - Functional         : Adding new module for node: {add_1} 
2023-01-16 17:54:15,008 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_1_relu_1} 
2023-01-16 17:54:15,008 - Quant - INFO - Functional         : Adding new module for node: {add_2} 
2023-01-16 17:54:15,009 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_0_relu_1} 
2023-01-16 17:54:15,010 - Quant - INFO - Functional         : Adding new module for node: {add_3} 
2023-01-16 17:54:15,010 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_1_relu_1} 
2023-01-16 17:54:15,011 - Quant - INFO - Functional         : Adding new module for node: {add_4} 
2023-01-16 17:54:15,011 - Quant - INFO - Reused/Duplicate   : Adding ne

/usr/local/lib/python3.8/dist-packages/torch/fx/graph.py:606: UserWarning: Attempted to insert a call_module Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule
  warnings.warn("Attempted to insert a call_module Node with "


In [11]:
_ = fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

2023-01-16 17:54:15,409 - Utils - INFO - ...... subset to store [Conv_0, BatchNormalization_1]
2023-01-16 17:54:15,410 - Utils - INFO - ...... subset to store [Conv_6, BatchNormalization_7]
2023-01-16 17:54:15,411 - Utils - INFO - ...... subset to store [Conv_11, BatchNormalization_12]
2023-01-16 17:54:15,411 - Utils - INFO - ...... subset to store [Conv_17, BatchNormalization_18]
2023-01-16 17:54:15,411 - Utils - INFO - ...... subset to store [Conv_22, BatchNormalization_23]
2023-01-16 17:54:15,412 - Utils - INFO - ...... subset to store [Conv_28, BatchNormalization_29]
2023-01-16 17:54:15,412 - Utils - INFO - ...... subset to store [Conv_33, BatchNormalization_34]
2023-01-16 17:54:15,413 - Utils - INFO - ...... subset to store [Conv_41, BatchNormalization_42]
2023-01-16 17:54:15,413 - Utils - INFO - ...... subset to store [Conv_46, BatchNormalization_47]
2023-01-16 17:54:15,413 - Utils - INFO - ...... subset to store [Conv_52, BatchNormalization_53]
2023-01-16 17:54:15,414 - Utils - 

In [12]:
dummy_input = torch.rand(1, 3, 224, 224)    # Shape for each ImageNet sample is (3 channels) x (224 height) x (224 width)
dummy_input = dummy_input.cuda()

sim = QuantizationSimModel(model=model,
#                            quant_scheme=QuantScheme.post_training_tf,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

2023-01-16 17:54:16,355 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.8/dist-packages/aimet_common/quantsim_config/default_config.json
2023-01-16 17:54:16,374 - Quant - INFO - Unsupported op type Squeeze
2023-01-16 17:54:16,375 - Quant - INFO - Unsupported op type Pad
2023-01-16 17:54:16,375 - Quant - INFO - Unsupported op type Mean
2023-01-16 17:54:16,382 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]
2023-01-16 17:54:16,382 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]
2023-01-16 17:54:16,383 - Utils - INFO - ...... subset to store [Conv_5, Relu_6]
2023-01-16 17:54:16,383 - Utils - INFO - ...... subset to store [Conv_5, Relu_6]
2023-01-16 17:54:16,384 - Utils - INFO - ...... subset to store [Add_10, Relu_11]
2023-01-16 17:54:16,384 - Utils - INFO - ...... subset to store [Add_10, Relu_11]
2023-01-16 17:54:16,385 - Utils - INFO - ...... subset to store [Conv_14, Relu_15]
2023-01-16 17:54:16,385 - Utils - INFO - .....

In [13]:
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=True)

500
1000


In [14]:
# post_training_tf_enchanced
accuracy(sim.model, test_loader, device='cuda')

100%|██████████| 100/100 [01:56<00:00,  1.17s/it]


0.44936

In [20]:
# post_training_tf
accuracy(sim.model, test_loader, device='cuda')

100%|██████████| 100/100 [01:45<00:00,  1.05s/it]


0.4189

# AdaRound

In [10]:
model = prepare_model(model)

2023-01-16 18:10:51,197 - Quant - INFO - Functional         : Adding new module for node: {add} 
2023-01-16 18:10:51,198 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_0_relu_1} 
2023-01-16 18:10:51,199 - Quant - INFO - Functional         : Adding new module for node: {add_1} 
2023-01-16 18:10:51,199 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_1_relu_1} 
2023-01-16 18:10:51,200 - Quant - INFO - Functional         : Adding new module for node: {add_2} 
2023-01-16 18:10:51,201 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_0_relu_1} 
2023-01-16 18:10:51,201 - Quant - INFO - Functional         : Adding new module for node: {add_3} 
2023-01-16 18:10:51,202 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_1_relu_1} 
2023-01-16 18:10:51,202 - Quant - INFO - Functional         : Adding new module for node: {add_4} 
2023-01-16 18:10:51,203 - Quant - INFO - Reused/Duplicate   : Adding ne

/usr/local/lib/python3.8/dist-packages/torch/fx/graph.py:606: UserWarning: Attempted to insert a call_module Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule
  warnings.warn("Attempted to insert a call_module Node with "


In [11]:
_ = fold_all_batch_norms(model, input_shapes=(1, 3, 224, 224))

2023-01-16 18:10:51,900 - Utils - INFO - ...... subset to store [Conv_0, BatchNormalization_1]
2023-01-16 18:10:51,901 - Utils - INFO - ...... subset to store [Conv_6, BatchNormalization_7]
2023-01-16 18:10:51,901 - Utils - INFO - ...... subset to store [Conv_11, BatchNormalization_12]
2023-01-16 18:10:51,902 - Utils - INFO - ...... subset to store [Conv_17, BatchNormalization_18]
2023-01-16 18:10:51,902 - Utils - INFO - ...... subset to store [Conv_22, BatchNormalization_23]
2023-01-16 18:10:51,902 - Utils - INFO - ...... subset to store [Conv_28, BatchNormalization_29]
2023-01-16 18:10:51,903 - Utils - INFO - ...... subset to store [Conv_33, BatchNormalization_34]
2023-01-16 18:10:51,904 - Utils - INFO - ...... subset to store [Conv_41, BatchNormalization_42]
2023-01-16 18:10:51,904 - Utils - INFO - ...... subset to store [Conv_46, BatchNormalization_47]
2023-01-16 18:10:51,904 - Utils - INFO - ...... subset to store [Conv_52, BatchNormalization_53]
2023-01-16 18:10:51,905 - Utils - 

In [12]:
params = AdaroundParameters(data_loader=val_loader, num_batches=2000//val_loader.batch_size, default_num_iterations=10000)

In [13]:
dummy_input = torch.rand(1, 3, 224, 224).cuda()
# adaround_path = f"adaround_e={eps}_d={decomp}_{qscheme.split('_')[-1]}.calibrated_{num_samples}"
adaround_path = 'adaround'

In [14]:
os.makedirs(adaround_path, exist_ok=True)

In [15]:
%time
dummy_input = torch.rand(1, 3, 224, 224)  
dummy_input = dummy_input.cuda()

ada_model = Adaround.apply_adaround(model, dummy_input, params,
                                    path=adaround_path, 
                                    filename_prefix='adaround', 
                                    default_param_bw=8,
                                    default_quant_scheme=QuantScheme.post_training_tf_enhanced)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs
2023-01-16 18:10:56,731 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.8/dist-packages/aimet_common/quantsim_config/default_config.json
2023-01-16 18:10:56,750 - Quant - INFO - Unsupported op type Squeeze
2023-01-16 18:10:56,750 - Quant - INFO - Unsupported op type Pad
2023-01-16 18:10:56,751 - Quant - INFO - Unsupported op type Mean
2023-01-16 18:10:56,757 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]
2023-01-16 18:10:56,758 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]
2023-01-16 18:10:56,758 - Utils - INFO - ...... subset to store [Conv_5, Relu_6]
2023-01-16 18:10:56,759 - Utils - INFO - ...... subset to store [Conv_5, Relu_6]
2023-01-16 18:10:56,759 - Utils - INFO - ...... subset to store [Add_10, Relu_11]
2023-01-16 18:10:56,759 - Utils - INFO - ...... subset to store [Add_10, Relu_11]
2023-01-16 18:10:56,760 - Utils - INFO - ...... subset to store [

2023-01-16 18:11:05,909 - Quant - INFO - Started Optimizing weight rounding of module: conv1


2023-01-16 18:12:10,295 - Quant - INFO - Started Optimizing weight rounding of module: layer1.0.conv1.conv1


2023-01-16 18:12:37,977 - Quant - INFO - Started Optimizing weight rounding of module: layer1.0.conv1.conv2


2023-01-16 18:13:04,232 - Quant - INFO - Started Optimizing weight rounding of module: layer1.0.conv1.conv3


...

2023-01-16 18:33:03,107 - Quant - INFO - Started Optimizing weight rounding of module: layer4.1.conv2.conv2


2023-01-16 18:33:24,214 - Quant - INFO - Started Optimizing weight rounding of module: layer4.1.conv2.conv3


2023-01-16 18:33:46,612 - Quant - INFO - Started Optimizing weight rounding of module: fc


100%|██████████| 100/100 [23:01<00:00, 13.82s/it]

2023-01-16 18:34:07,795 - Quant - INFO - Deleting model inputs from location: /tmp/adaround/
2023-01-16 18:34:07,905 - Quant - INFO - Completed Adarounding Model


In [18]:
dummy_input = torch.rand(1, 3, 224, 224)    
dummy_input = dummy_input.cuda()

sim = QuantizationSimModel(model=ada_model,
#                            quant_scheme=QuantScheme.post_training_tf,
                           quant_scheme=QuantScheme.post_training_tf_enhanced,
                           dummy_input=dummy_input,
                           default_output_bw=8,
                           default_param_bw=8)

2023-01-16 18:40:17,727 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.8/dist-packages/aimet_common/quantsim_config/default_config.json
2023-01-16 18:40:17,745 - Quant - INFO - Unsupported op type Squeeze
2023-01-16 18:40:17,746 - Quant - INFO - Unsupported op type Pad
2023-01-16 18:40:17,746 - Quant - INFO - Unsupported op type Mean
2023-01-16 18:40:17,753 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]
2023-01-16 18:40:17,753 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]
2023-01-16 18:40:17,754 - Utils - INFO - ...... subset to store [Conv_5, Relu_6]
2023-01-16 18:40:17,754 - Utils - INFO - ...... subset to store [Conv_5, Relu_6]
2023-01-16 18:40:17,755 - Utils - INFO - ...... subset to store [Add_10, Relu_11]
2023-01-16 18:40:17,755 - Utils - INFO - ...... subset to store [Add_10, Relu_11]
2023-01-16 18:40:17,756 - Utils - INFO - ...... subset to store [Conv_14, Relu_15]
2023-01-16 18:40:17,756 - Utils - INFO - .....

In [19]:
sim.set_and_freeze_param_encodings(encoding_path=os.path.join(adaround_path, 'adaround.encodings'))

2023-01-16 18:40:21,479 - Quant - INFO - Setting quantization encodings for parameter: conv1.weight
2023-01-16 18:40:21,479 - Quant - INFO - Freezing quantization encodings for parameter: conv1.weight
2023-01-16 18:40:21,480 - Quant - INFO - Setting quantization encodings for parameter: layer1.0.conv1.conv1.weight
2023-01-16 18:40:21,480 - Quant - INFO - Freezing quantization encodings for parameter: layer1.0.conv1.conv1.weight
2023-01-16 18:40:21,481 - Quant - INFO - Setting quantization encodings for parameter: layer1.0.conv1.conv2.weight
2023-01-16 18:40:21,481 - Quant - INFO - Freezing quantization encodings for parameter: layer1.0.conv1.conv2.weight
2023-01-16 18:40:21,481 - Quant - INFO - Setting quantization encodings for parameter: layer1.0.conv1.conv3.weight
2023-01-16 18:40:21,482 - Quant - INFO - Freezing quantization encodings for parameter: layer1.0.conv1.conv3.weight
2023-01-16 18:40:21,482 - Quant - INFO - Setting quantization encodings for parameter: layer1.0.conv2.conv

2023-01-16 18:40:21,513 - Quant - INFO - Setting quantization encodings for parameter: layer3.1.conv2.conv1.weight
2023-01-16 18:40:21,513 - Quant - INFO - Freezing quantization encodings for parameter: layer3.1.conv2.conv1.weight
2023-01-16 18:40:21,515 - Quant - INFO - Setting quantization encodings for parameter: layer3.1.conv2.conv2.weight
2023-01-16 18:40:21,516 - Quant - INFO - Freezing quantization encodings for parameter: layer3.1.conv2.conv2.weight
2023-01-16 18:40:21,516 - Quant - INFO - Setting quantization encodings for parameter: layer3.1.conv2.conv3.weight
2023-01-16 18:40:21,517 - Quant - INFO - Freezing quantization encodings for parameter: layer3.1.conv2.conv3.weight
2023-01-16 18:40:21,518 - Quant - INFO - Setting quantization encodings for parameter: layer4.0.conv1.conv1.weight
2023-01-16 18:40:21,518 - Quant - INFO - Freezing quantization encodings for parameter: layer4.0.conv1.conv1.weight
2023-01-16 18:40:21,519 - Quant - INFO - Setting quantization encodings for 

In [20]:
sim.compute_encodings(forward_pass_callback=pass_calibration_data,
                      forward_pass_callback_args=True)

500
1000


In [21]:
# tf_enchanced
accuracy(sim.model, test_loader, device='cuda')

100%|██████████| 100/100 [01:55<00:00,  1.15s/it]


0.59534

In [22]:
# tf
accuracy(sim.model, test_loader, device='cuda')

100%|██████████| 100/100 [01:46<00:00,  1.06s/it]


0.47252